# Creating a Vivado HLS Core


This notebook will walk through the process of creating a Vivado HLS core. This notebook assumes that you are familiar with Vivado HLS and have already used Vivado HLS to write, simluate, and debug a C/C++ core for hardware.


If you have not already, add Vivado HLS to your executable path. In Cygwin, you can do this by running:

``` bash
    source C:/Xilinx/Vivado/2017.1/settings64.sh
```

Or on Linux: 

``` bash
    source /opt/Xilinx/Vivado/2017.1/settings64.sh
```

These command assume that Vivado has been installed in `C:/Xilinx/Vivado` or `/opt/Xilinx/Vivado/`. If that is not the case, you should modify the commands above to match your installation path. 

This notebook assumes that you have cloned the [PYNQ-HLS repository](https://github.com/drichmond/PYNQ-HLS) to the home directory (`~`) on your computer. On our computer, this is the `/home/xilinx/` directory. 

To skip this notebook, run the following commands on your host computer: 

``` bash
     cp ~/PYNQ-HLS/pynqhls/io/ip/ctrlloop/* ~/PYNQ-HLS/tutorial/pynqhls/io/ip/ctrlloop/
     make -C ~/PYNQ-HLS/pynqhls/io/ip/ctrlloop/
```

## Objectives: 

We will be creating a real-time control loop as a High-Level Synthesis core. The loop will execute with a predictable period and interact with the physical world: Read buttons, set LED values, and print UART data.

This notebook will teach you how to : 
1. Create an AXI-Lite Interface for HLS Core configuration
2. Create a protocol-less Output Bus for driving LEDs
3. Create a protocol-less Input Bus for reading buttons
4. Create an AXI-Master Interface for communicating with an AXI peripheral

The AXI Lite interface will be connected to the ZYNQ ARM PS, the Input and Output busses will be connected to pins, and the AXI-Master Interface will be connected to an AXI UART Core.

## Creating a Vivado HLS Project


We will begin by creating a Vivado HLS project. On your host computer, navigate to the following folder of the PYNQ-HLS repository using your terminal:

```bash
    cd ~/PYNQ-HLS/tutorial/pynqhls/io/ip/ctrlloop/
```

In this directory we have provided a makefile that will: 

1. Create a `ctrlloop` directory with a Vivado HLS project
2. Add `ctrlloop.cpp`, `ctrlloop.hpp`, and `main.cpp` files to the project
3. Run tests for the `ctrlloop.cpp` file (**These will fail initially**)
4. If the tests pass, synthesize the core.

To run the makefile, run make from your current directory:

``` bash
    make
```

This will build the Vivado HLS project, but the testbench will fail because the method `ctrlloop` is not implemented. Open the project in the Vivado HLS tool by running the command:

```bash
    vivado_hls -p ~/PYNQ-HLS/tutorial/pynqhls/io/ip/ctrlloop/
```

This will open the following window:

<img src="pictures/vivadohls_ctrlloop_splash.png" alt="Ctrlloop Project in Vivado HLS" style="width: 768px;"/>

## Writing Your Core

The next step is to implement the `ctrlloop` core. Open the file `ctrlloop.cpp`. You will see the following method body: 

``` C++

#include "ctrlloop.hpp"
#include "ap_utils.h"

/* ctrlloop - A control loop function. Using the AP_AUTORESTART bit of the
 * AP_CTRL register this loop implements a real-time control loop.
 *
 * mem - ap_uint<32> - An array representing the peripheral memory space. This
 * will become an AXI-Master bus through the use of pragmas
 *
 * regs - ap_uint<32> - An array representing the internal memory space
 * (registers) of the core. The function can read and write to locations
 * here. This will be accessible from the ARM core using the CTRL AXI-Slave bus
 *
 * buttons - const ap_uint<4> - An ap_uint representing the current value on the
 * 4 buttons switches on the PYNQ board. The const specifier will make this
 * default to a 4-bit input port on the HLS core
 * 
 * leds - ap_uint<4>& - An ap_uint for assigning values to the LEDs. Using
 * pragmas this will become a 4-bit output port
 */
void ctrlloop(ap_uint<32> iomem [IOMEM_SPACE_SIZE],
	ap_uint<32> regs[REG_SPACE_SIZE],
	const ap_uint<4> buttons,
	ap_uint<4>& leds){

	// Write your code here!
}


```

As you can see, the body of the function `ctrlloop` is blank - **this is okay**. To pass the testbench we have provided in `main.cpp` you will need to fill out the functionality. 

To pass the testbench, you will need to: 

- Maintain the state of `leds` across executions (Hint, use a `static` variable)
- Increment the state of `leds` every time the code is executed
- Assign the argument `leds` when executed
- Read the value of `buttons` and store it in `regs[0]`
- Send `Hello World` using writes to the IO Memory Interface to an AXI UART Controller described in the [AXI Uartlite User Guide](https://www.xilinx.com/support/documentation/ip_documentation/axi_uartlite/v2_0/pg142-axi-uartlite.pdf)

To implement the core in hardware you will need to create the following interfaces: 

- AXI Master on the `iomem` argument
- AXI-Lite Slave on the control registers (Also known as `return`)
- No-Protocol on the `leds` argument
- No-Protocol on the `buttons` argument

To will need to maintain the following behavior:

- Delay until 1 second has elapsed (1,000,000 cycles at 100 MHz)

### Our Implementation

You can define your own implementation, or fill `ctrlloop.cpp` with the implementation below:

``` C++
#include "ctrlloop.hpp"
#include "ap_utils.h"

/* ctrlloop - A control loop function. Using the AP_AUTORESTART bit of the
 * AP_CTRL register this loop implements a real-time control loop.
 *
 * mem - ap_uint<32> - An array representing the peripheral memory space. This
 * will become an AXI-Master bus through the use of pragmas
 *
 * regs - ap_uint<32> - An array representing the internal memory space
 * (registers) of the core. The function can read and write to locations
 * here. This will be accessible from the ARM core using the CTRL AXI-Slave bus
 *
 * buttons - const ap_uint<4> - An ap_uint representing the current value on the
 * 4 buttons switches on the PYNQ board. The const specifier will make this
 * default to a 4-bit input port on the HLS core
 * 
 * leds - ap_uint<4>& - An ap_uint for assigning values to the LEDs. Using
 * pragmas this will become a 4-bit output port
 */
void ctrlloop(ap_uint<32> iomem [IOMEM_SPACE_SIZE],
	ap_uint<32> regs[REG_SPACE_SIZE],
	const ap_uint<4> buttons,
	ap_uint<4>& leds){
/* Define a new AXI-Lite bus named CTRL for HLS Status/Control registers
   (return)*/
#pragma HLS INTERFACE s_axilite port=return bundle=CTRL
/* Define the argument reg as an AXI-Slave port, shared with the bundle
 * CTRL. Values read and written to reg will be accessible from this AXI-Slave
 * bus. The address of the registers is equal to REG_SPACE_SIZE (by default) */
#pragma HLS INTERFACE s_axilite port=regs   bundle=CTRL
/* Define a new AXI-Master bus named MEM represented by the argument
 * iomem. Writes and reads to iomem will be seen on this AXI port. This bus will
 * be used to talk to AXI peripherals such as UART, SPI, and I2C */ 
#pragma HLS INTERFACE m_axi port=iomem bundle=IOMEM
/* Define the port leds to be a port with no protocol. This will generate a
 * 4-bit output port for driving leds*/
#pragma HLS INTERFACE ap_none port=leds
/* Define the port buttons to be a port with no protocol. This will generate a
 * 4-bit input port for reading the buttons*/
#pragma HLS INTERFACE ap_none port=buttons

/* Declare the LED State as a static ap_uint<4> */
	static ap_uint<4> led_state = 0;
    
/* Define that the static variable led_state should be driven to 0 (it's default
 * value) on reset */
#pragma HLS reset variable=led_state
	led_state++;
/* Drive the LEDs to the current value of led_state */
	leds = led_state;

/* Set the value of reg at index 0 to the value of buttons. */
	regs[0] = buttons;

/* Make this function delay until 1000 milliseconds have passed since
 * it started */
	delay_until_ms<1000>();	
	return;
}

```

### Sub-Functions

This relies on two sub-functions: `delay_until_ms` and `write_uart`. We define the `delay_until_ms` function in `ctrlloop.hpp` with the implementation below: 

``` C++

/* delay_until_ms - Delay for a set number of milliseconds. The length of the
   delay is equal to the frequency of the overlay (F_OVERLAY_HZ) times the
   duration in milliseconds (MILLISECONDS), divided by the number of
   milliseconds in a second (MSEC_PER_SEC)
 */
template <unsigned int MILLISECONDS, unsigned int F_OVERLAY_HZ = 50000000ULL>
void delay_until_ms(){
#define MSEC_PER_SEC 1000
#pragma HLS INLINE
#pragma HLS PROTOCOL floating
	volatile char dummy;
	ap_uint<64> ctr;
	ap_uint<64> cyc = (F_OVERLAY_HZ * MILLISECONDS / MSEC_PER_SEC);
	for (ctr = 0; ctr < cyc; ++ctr){
		dummy = dummy;
	}
	return;
#undef MSEC_PER_SEC
}

```

`write_uart` writes a string of data to the AXI UART Controller we will add to the bitstream in the next notebook. Data is written with a sequence of writes and reads to register locations. These registers and the sequence can be found in the [AXI Uartlite User Guide](https://www.xilinx.com/support/documentation/ip_documentation/axi_uartlite/v2_0/pg142-axi-uartlite.pdf)


``` C++

/* write_uart -
 */
void write_uart(ap_uint<32> regs[REG_SPACE_SIZE], 
                const char* str){
#pragma HLS INLINE off
}

```

# Interfaces

### AXI-Master Interface

The AXI-Master interface on `iomem` is declared using the pragma below: 

``` C

#pragma HLS INTERFACE m_axi port=iomem bundle=IOMEM

```

This defines a AXI-Master interface that will be named `m_axi_IOMEM` in Vivado. AXI-Master is a memory-mapped bus interface driven by the HLS Core. Accesses on the `iomem` array will appear as reads/writes byte addresses on the `m_axi_IOMEM` bus interface of the hardware core. We will use this interface to read and write to AXI-Slave peripherals, like an AXI-UART Controller in this example. 


### AXI-Lite Interfaces

AXI-Lite interfaces are declared using the following pragmas: 

``` C

#pragma HLS INTERFACE s_axilite port=return bundle=CTRL
#pragma HLS INTERFACE s_axilite port=regs   bundle=CTRL

```
This defines a AXI-Lite interface that will be named `s_axi_CTRL` in Vivado. AXI Lite is used for configuration data. It is low-performance and uses few resources. The `pragma` in the first line above defines an AXI-Lite interface for the control registers, called the `return` argument. The second line defines an AXI-Lite interface for the `regs` array. This allows the Zynq ARM PS to issue reads to indices in the array over the AXI-Lite interface. These two interfaces are combined by providing the same name to the `bundle` argument. 


### No-Protocol Interfaces

The No-Protocol Interfaces are defined using the following pragmas: 

``` C
#pragma HLS INTERFACE ap_none port=leds
#pragma HLS INTERFACE ap_none port=buttons

```

This defines two No-Protcol interfaces. They will be named `buttons_V` and `leds_V`. No-Protocol interfaces provide bare wires for direct input and output. The use of `const` on the argument `buttons` tells HLS to create a No-Protocol input interface (no writes), and the absence of `const` on the argument `leds` tells HLS to create a No-Protocol Output interface. 


 ## Compiling

Once you have filled the implementation, click the **Run C Simulation ** and then **Synthesize** button. This will produce the window shown below: 

<img src="pictures/vivadohls_ctrlloop_synth.png" alt="Synthesized ctrlloop function in Vivado HLS" style="width: 768px;"/>

In the center window scroll down to view the ports. The ports list shows us the interface signals for our HLS core. The port names are unimportant in this example - what matters is the protocol field. The protocol field has four types: **s_axi_lite (AXI Lite, Slave Interface)**, **m_axi (AXI Master Interface)**, and **ap_ctrl_hs (Control Signals)**, and **ap_none (No-Protocol Interface)**. 

For best results in Vivado (and PYNQ) your core should provide **s_axi_lite**, **axis**, or **m_axi** interfaces for data transfer. These ports are automagically recognized by Vivado and can be used in the Block Diagram editor in the **[Building a Bitstream](3-Building-A-Bitstream.ipynb)** notebook.

**ap_ctrl_hs** signals provide clock, reset, and interrupt ports.

**ap_none** signals are useful for No-Protocol interfaces like GPIO.

When the process has completed, save, and exit Vivado HLS. 

## Testing and Recompiling the Core

Once this process has been completed, you can re-compile the HLS core and run the tests by executing the following commands:

```bash
    cd ~/PYNQ-HLS/pynqhls/io/ip/ctrlloop
    make clean ctrlloop
```

If the tests pass, proceed to the **[Building a Bitstream](3-Building-A-Bitstream.ipynb)** notebook.